# Explorando IA Generativa em um Pipeline de ETL com Python

## Instruções iniciais

[Repositório no GitHub da API](https://github.com/digitalinnovationone/santander-dev-week-2023-api)


A partir do Swagger da API, foi criado 4 IDs para usar nesse desafio.

Passos:
1. Criação de 4 IDs (apenas informações de nome, conta, agencia, dinheiro - o ID o próprio API criou).
2. Criação de um arquivo em excel com os números dos IDs criados.
3. Arquivo upado aqui para extração no pandas.

In [ ]:
# API
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## Extração

Extraçao dos IDs:

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('SDW2023.xlsx')

In [ ]:
user_ids = df['UserID'].tolist()

Obtenção dos dados de cada ID usando a API do Santander Dev Week 2023 (dando GET nos endpoints):

In [ ]:
import requests
import json

In [ ]:
# Dando GET nos endpoints
def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

In [ ]:
users = [user for id in user_ids if (user := get_user(id)) is not None]

In [ ]:
print(json.dumps(users, indent=2))

[
  {
    "id": 1528,
    "name": "Joao",
    "account": {
      "id": 1601,
      "number": "59601-1",
      "agency": "3601",
      "balance": 0.0,
      "limit": 8900.0
    },
    "card": {
      "id": 1462,
      "number": "**** **** **** 1635",
      "limit": 36900.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1529,
    "name": "Silvana",
    "account": {
      "id": 1602,
      "number": "57661-2",
      "agency": "3271",
      "balance": 0.0,
      "limit": 800.0
    },
    "card": {
      "id": 1463,
      "number": "**** **** **** 6358",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1530,
    "name": "Duda",
    "account": {
      "id": 1603,
      "number": "36740-2",
      "agency": "4970",
      "balance": 0.0,
      "limit": 1530.0
    },
    "card": {
      "id": 1464,
      "number": "**** **** **** 2014",
      "limit": 3000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 1531,
    "name": "Fantasma",


## Transform

Utilizando a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Chave da API
openai_api_key = 'xx'

In [ ]:
import openai

In [ ]:
openai.api_key = openai_api_key

In [ ]:
def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário." # o que o bot será
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)" # comando pro bot executar
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

In [ ]:
for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Olá João, invista para garantir seu futuro financeiro. Seu dinheiro trabalhando por você!
Silvana, investir é essencial para construir riqueza e segurança financeira. Vamos começar?
Duda, investir é crucial. Faz seu dinheiro trabalhar para você, garantindo futuro financeiro estável.
Fantasma, invista hoje para um futuro seguro e próspero. Faça o seu dinheiro trabalhar para você!


## Load

Atualizando a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

In [ ]:
for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Joao updated? True!
User Silvana updated? True!
User Duda updated? True!
User Fantasma updated? True!
